In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
import time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from mpl_toolkits.mplot3d import Axes3D

In [2]:
"""
Author: Naman Krishna Pande
Built upon Dr. Maziar Raissi's PINNs - https://github.com/maziarraissi/PINNs

Use Tensorflow 1.15

"""

se = 1234
np.random.seed(se)
tf.set_random_seed(se)
u_f = 80
rho_m = 0.25
tau = 10
c0 = 20

class PhysicsInformedNN:
    def __init__(self, X_u, rho, u, X_f, layers, lb, ub):

        self.lb = lb
        self.ub = ub
        self.x_u = X_u[:,0:1]
        self.x_f = X_f[:,0:1]
        self.t_u = X_u[:,1:2]
        self.t_f = X_f[:,1:2]
        self.u = u
        self.rho = rho
        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))

        self.x_u_tf = tf.placeholder(tf.float32, shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.placeholder(tf.float32, shape=[None, self.t_u.shape[1]])
        self.u_tf   = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.rho_tf = tf.placeholder(tf.float32, shape=[None, self.rho.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])
        self.Y_pred = self.net_u(self.x_u_tf, self.t_u_tf)
        self.rho_pred = self.Y_pred[:,0:1]
        self.u_pred = self.Y_pred[:,1:2]
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        self.f1 = self.f_pred[0]
        self.f2 = self.f_pred[1]

        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + 1.5*tf.reduce_mean(tf.square(self.rho_tf - self.rho_pred)) + tf.reduce_mean(tf.square(self.f1)) + tf.reduce_mean(tf.square(self.f2))
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                                method = 'L-BFGS-B',
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' :1.0 * np.finfo(float).eps})
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0, num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        ##################### Addition ##############################
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, seed=se), dtype=tf.float32)
        #############################################################

    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2*(X - self.lb)/(self.ub - self.lb) - 1
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = 1-tf.tanh(tf.add(tf.matmul(H, W), b))**2
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)

        return Y

    def net_u(self, x, t):
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        return u

    def net_f(self, x,t):
        NN = self.net_u(x,t)
        rho = NN[:,0:1]
        u = NN[:,1:2]
#         U_eq = u_f*(1-tf.exp(1-tf.exp(cm/u_f*(rho_m/rho-1))))
        U_eq = u_f*((1+tf.exp((rho/rho_m-0.25)/0.06))**-1)-u_f*372*10**-8
        drho_t = tf.gradients(rho,t)[0]
        drhou_x = tf.gradients(rho*u,x)[0]
        du_t = tf.gradients(u,t)[0]
        du_x = tf.gradients(u,x)[0]
        f1 = drho_t+drhou_x
        f2 = du_t+u*du_x-(U_eq-u)/tau-c0*du_x
        return [f1,f2]

    def callback(self, loss):
        print('Loss:', loss)

    def train(self):
        tf_dict = {self.x_u_tf: self.x_u, self.t_u_tf: self.t_u,
                   self.u_tf: self.u,self.rho_tf: self.rho,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)

    def predict(self, X_star):
        Y_star = self.sess.run(self.Y_pred, {self.x_u_tf: X_star[:,0:1], self.t_u_tf: X_star[:,1:2]})
        f_star = self.sess.run(self.f_pred, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})
        return Y_star, f_star
    
    

layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 2]
datarho = pd.read_csv('NGSIM_US101_Density_Data.csv')
datau = pd.read_csv('NGSIM_US101_Velocity_Data.csv')
Exactrho = np.real(datarho).T
Exactu = np.real(datau).T
num = 1
N_u = (540*(num+2)+104)
N_f = 20000
uxn = 104
xlo = 0.
xhi = 2060

utn = 540
tlo = 0.
thi = 2695
x = np.linspace(xlo, xhi, uxn)
t = np.linspace(tlo, thi, utn)

X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
rho_star = Exactrho.flatten()[:, None]
u_star = Exactu.flatten()[:, None]
######################## Eulerian Training Data #################################

xx1 = np.hstack((X[0:1, :].T, T[0:1, :].T))
xx2 = np.hstack((X[:, 0:1], T[:, 0:1]))
xx3 = np.hstack((X[:, -1:], T[:, -1:]))
xx4 = np.hstack((X[:, 15:16], T[:, 15:16]))
xx5 = np.hstack((X[:, 85:86], T[:, 85:86]))
xx6 = np.hstack((X[:, 30:31], T[:, 30:31]))
xx7 = np.hstack((X[:, 60:61], T[:, 60:61]))

rho1 = Exactrho[0:1, :].T
rho2 = Exactrho[:, 0:1]
rho3 = Exactrho[:,-1:]
rho4 = Exactrho[:, 15:16]
rho5 = Exactrho[:, 85:86]
rho6 = Exactrho[:, 30:31]
rho7 = Exactrho[:, 60:61]

u1 = Exactu[0:1, :].T
u2 = Exactu[:, 0:1]
u3 = Exactu[:,-1:]
u4 = Exactu[:, 15:16]
u5 = Exactu[:, 85:86]
u6 = Exactu[:, 30:31]
u7 = Exactu[:, 60:61]
X_u_train1 = np.vstack([xx1, xx2, xx3, xx5])#xx4])#, xx5, xx6, xx7])
rho_train1 = np.vstack([rho1, rho2, rho3, rho5])#rho4])#, rho5, rho6, rho7])
u_train1 = np.vstack([u1, u2, u3, u5])# u4])#, u5, u6, u7])
idx1 = np.random.choice(X_u_train1.shape[0], N_u, replace=False)
X_u_train = X_u_train1[idx1, :]
u_train = u_train1[idx1, :]
rho_train = rho_train1[idx1,:]

############################## Collocation Points ################################

lb = X_star.min(0)
ub = X_star.max(0)
X_f_train = lb + (ub - lb) * lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))


############################### Model Training ##################################
model = PhysicsInformedNN(X_u_train, rho_train, u_train, X_f_train, layers, lb, ub)
start_time = time.time()
model.train()
elapsed = time.time() - start_time
print('Training time: %.4f' % elapsed)
Y_pred, f_pred = model.predict(X_star)
u = Y_pred[:,1:2]
rho = Y_pred[:,0:1]
U_pred = griddata(X_star, u.flatten(), (X, T), method='cubic')
rho_pred = griddata(X_star, rho.flatten(), (X, T), method='cubic')

In [ ]:
t1 = np.linspace(0, 2690, 540)
x1 = np.linspace(0, 2060, 104)
x, t = np.meshgrid(x1, t1) 
plt.figure()
z = density
# Creating the top view contour plot
fig, ax = plt.subplots(figsize = (15,7))
im = ax.imshow(z.T, extent=[t1.min(), t1.max(), x1.min(), x1.max()], cmap= 'seismic', origin= 'lower', aspect = 'auto')
# plt.xlim(-0.01,3)
# plt.title('Top View Contour Plot of sin(X) * cos(Y)')
plt.xlabel(r'Time ($t$)', fontsize = 45, rotation = 0, labelpad = 20)
plt.ylabel(r'Location ($x$)', fontsize = 45, rotation = 90, labelpad = 20)
plt.xticks(np.round([0, 2690//2, 2690], 2), fontsize = 35)
plt.yticks(np.round([2060//2, 2060], 2), fontsize = 35)
plt.gca().yaxis.get_offset_text().set_fontsize(30)  # Adjust the font size here
plt.gca().xaxis.get_offset_text().set_fontsize(30)  # Adjust the font size here
plt.text(x=2690//2-5, y = 2200, s=r'Density ($\rho$)', fontsize=45, ha='center', va='center')
plt.grid(False)
fig.add_axes(ax)
cax = fig.add_axes([0.125, -0.15, 0.78, 0.03])  # [left, bottom, width, height]
cbar = fig.colorbar(surface, cax=cax, orientation='horizontal')
cbar.ax.tick_params(labelsize=30)  # Adjust the labelsize parameter as needed
plt.show()

In [ ]:
plt.figure(figsize = (20,15))
k = 1
for j in [0, 150, 300, 450]:
    plt.subplot(2,2,k)
    plt.subplots_adjust(hspace=0.3)# Adjust hspace value as needed for padding)
    plt.plot(x1, dens.T[j], linewidth = 8, color = '#02066F', alpha = 0.75, label = 'True')
    plt.plot(x1, density[j], linestyle = 'dashed', linewidth = 4, color = 'red', label = 'Reconstruction')
    plt.xticks([0, 2060//2, 2060], fontsize = 40)
    plt.yticks(np.round(np.linspace(0, 0.25, 3), 2),fontsize = 40)
    plt.ylim(0, 0.25)
    plt.gca().yaxis.get_offset_text().set_fontsize(30)  # Adjust the font size here
    plt.gca().xaxis.get_offset_text().set_fontsize(30)
    if j == 0:
#         plt.yticks([0, 0.40, 0.80, 1.20], fontsize = 40)
        leg = plt.legend(loc='upper left', fontsize = 40, frameon = True, bbox_to_anchor=(0.35, -1.6), ncol= 3, edgecolor ='black')
        leg.get_frame().set_linewidth(1)
        leg.get_frame().set_edgecolor('black')
#     else:
#         plt.ylim(0.85, 1.1)
#         plt.yticks(np.round(np.linspace(0.85, 1.1, 3), 2), fontsize = 40)
    if j == 0 or j == 300:
        plt.ylabel(r'Velocity ($v$)', fontsize = 40, rotation = 90, labelpad = 10)
    if j == 300 or j == 450:
        plt.xlabel(r'Location ($x$)', fontsize = 40)
    plt.title(r'Time $(t)=$'+str(np.round(j*dt, 2))+' sec', fontsize = 40, pad = 0)
#     plt.ylim(0,1) 
      # Adjust the position of the title
    k+=1
plt.show()